In [9]:
import openml
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import pandas as pd
import numpy as np
import scipy.stats as ss
import scikit-posthocs as sp
import matplotlib.pyplot as plt

SyntaxError: invalid syntax (4133996220.py, line 12)

In [ ]:
def run_benchmark(models, model_names, benchmark="OpenML-CC18"):
    results = pd.DataFrame(columns=["dataset", "model", "score"]) # create dataframe for results
    benchmark_suite = openml.study.get_suite(benchmark) # obtain the benchmark suite

    subset_benchmark_suite = benchmark_suite.tasks[0:10] # subset tasks

    #for task_id in benchmark_suite.tasks:  # iterate over all tasks
    for task_id in subset_benchmark_suite: # iterate over subset tasks
        task = openml.tasks.get_task(task_id)  # download the OpenML task
        features, targets = task.get_X_and_y()  # get the data
        for model in range(len(models)): # iterate over all models
            score = np.mean(cross_val_score(models[model], features, targets, cv=10, scoring="roc_auc_ovr")) # calculate the metric for kfold
            if model_names:
                results = pd.concat([results, pd.DataFrame([[task_id, model_names[model], score]], columns=results.columns)], ignore_index=True) # insert the metric for a models and a dataset
            else:
                results = pd.concat([results, pd.DataFrame([[task_id, str(models[model]), score]], columns=results.columns)], ignore_index=True) # insert the metric for a models and a dataset
    results.to_csv("results.csv", index=False)

In [ ]:
KNN = make_pipeline(SimpleImputer(strategy='constant'),StandardScaler(),KNeighborsClassifier())
LR = make_pipeline(SimpleImputer(strategy='constant'),StandardScaler(),LogisticRegression())
models = [DecisionTreeClassifier(), KNN, LR]
model_names = ["DT","KNN","LR"]
run_benchmark(models=models, model_names=model_names)

In [ ]:
results = pd.read_csv("results.csv")
avg_rank = results.groupby('dataset').score.rank(pct=True).groupby(results.model).mean()
avg_rank

In [ ]:
results

In [ ]:
ss.friedmanchisquare(results.loc[results["model"]=="DT",:][["score"]], results.loc[results["model"]=="LR",:][["score"]], results.loc[results["model"]=="KNN",:][["score"]])

In [ ]:
test_results = sp.posthoc_conover_friedman(
    results,
    melted=True,
    block_col='dataset',
    group_col='model',
    y_col='score',
)
sp.sign_plot(test_results)

In [ ]:
plt.figure(figsize=(10, 2), dpi=100)
plt.title('Critical difference diagram of average score ranks')
sp.critical_difference_diagram(avg_rank, test_results)